In [99]:
%matplotlib inline
from __future__ import print_function

In [172]:
import pandas as pd
from sklearn.ensemble import AdaBoostClassifier, GradientBoostingClassifier, ExtraTreesClassifier
from sklearn.svm import SVC
#from sklearn.cross_validation import cross_val_score, KFold, StratifiedKFold
from sklearn.metrics import confusion_matrix
from scipy.stats import sem
import math
import numpy as np
import matplotlib.pyplot as plt

# Contains linear models, e.g., linear regression, ridge regression, LASSO, etc.
import sklearn.linear_model as sklin
# Provides train-test split, cross-validation, etc.
import sklearn.cross_validation as skcv
# Provides grid search functionality
import sklearn.grid_search as skgs
# For data normalization
import sklearn.preprocessing as skpr
# Allows us to create custom scoring functions
import sklearn.metrics as skmet

In [101]:
def score(gtruth, pred):
    diff = gtruth - pred
    return np.sqrt(np.mean(np.square(diff)))

In [102]:
def eval_pred(predictor, X, y, K=3):
    #ret = {}
    #scores = cross_val_score(pred, X, y, cv = 3)
    #print (scores)
    #print ("CV score: %.4f +/- %.4f" % (np.mean(scores), sem(scores)))  
    scorefun = skmet.make_scorer(score)
    scores = skcv.cross_val_score(predictor, X, y, scoring=scorefun, cv=10)
    print('C-V score =', np.mean(scores), '+/-', np.std(scores))

# [read file]

In [228]:
X = pd.read_csv('train.csv')
#X.names=['id', 'y', 'x1','x2', 'x3', 'x4', 'x5', 'x6', 'x7', 'x8', 'x9', 'x10', 'x11', 'x12', 'x13', 'x14', 'x15'])
#del X['id']
#y = X.cls
#del X['cls']

In [229]:
del X['Id']
y = X.y
del X['y']
pd.DataFrame(X).head()

,x1,x2,x3,x4,x5,x6,x7,x8,x9,x10,x11,x12,x13,x14,x15
0,2.764052,1.400157,1.978738,3.240893,2.867558,0.022722,1.950088,0.848643,0.896781,1.410599,1.144044,2.454274,1.761038,1.121675,1.443863
1,1.333674,2.494079,0.794842,1.313068,0.145904,-1.552990,1.653619,1.864436,0.257835,3.269755,-0.454366,1.045759,0.812816,2.532779,2.469359
2,1.154947,1.378163,0.112214,-0.980796,0.652088,1.156349,2.230291,2.202380,0.612673,0.697697,-0.048553,-0.420018,-0.706270,2.950775,0.490348
3,0.561926,-0.252795,1.777490,-0.613898,0.787260,0.104533,1.386902,0.489195,-0.180632,0.971818,1.428332,1.066517,1.302472,0.365678,0.637259
4,0.327540,0.640447,0.186854,-0.726283,1.177426,0.598219,-0.630198,1.462782,0.092702,1.051945,1.729091,1.128983,2.139401,-0.234826,1.402342


In [230]:
Xtest = pd.read_csv('test.csv')
del Xtest['Id']

In [231]:
X.shape, y.shape, Xtest.shape

((900, 15), (900,), (2000, 15))

In [232]:
sub = pd.read_csv('sample.csv')
#sub.head()

Normalization

In [233]:
from sklearn.preprocessing import normalize

In [234]:
XX = X.append(Xtest)
XX = normalize(XX)
XX.shape, X.shape

((2900, 15), (900, 15))

In [235]:
#pd.DataFrame(XX).head()

In [236]:
Xn = XX[:X.shape[0],:]
Xtn = XX[X.shape[0]:,:]

In [237]:
#pd.DataFrame(Xn).head()

In [238]:
#pd.DataFrame(Xtn).head()

Train Classifier

In [239]:
Xn.shape, y.shape

((900, 15), (900,))

In [240]:
#pd.DataFrame(Xn).head()

In [241]:
pd.DataFrame(y).head()

,y
0,56.140939
1,67.986654
2,105.711484
3,14.703316
4,8.868234


# [correlations of variables with prediction ]

In [403]:
XYtrain = np.vstack((Xn.T, np.atleast_2d(y)))
correlations = np.corrcoef(XYtrain)[-1, :]
feature_names=['x1','x2', 'x3', 'x4', 'x5', 'x6', 'x7', 'x8', 'x9', 'x10', 'x11', 'x12', 'x13', 'x14', 'x15']
feature_sel = []
print('features', len(feature_names), 'correlations', len(correlations))
i = 0
feature_vec = np.zeros(15)
for feature_name, correlation in zip(feature_names, correlations):
    if abs(correlation) > 0.02:
        feature_sel.append(feature_name)
        feature_vec[i] = 1
    print('{0:>10} {1:+.4f}'.format(feature_name, correlation))
    i += 1
print('{0:>10} {1:+.4f}'.format('OUTPUT', correlations[-1]))

features 15 correlations 16
        x1 +0.1031
        x2 +0.0001
        x3 -0.0887
        x4 +0.0327
        x5 -0.0485
        x6 +0.1429
        x7 +0.2581
        x8 +0.0219
        x9 -0.0245
       x10 -0.1692
       x11 -0.0265
       x12 +0.1708
       x13 -0.2787
       x14 +0.3736
       x15 +0.1323
    OUTPUT +1.0000


In [404]:
feature_vec

array([ 1.,  0.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,
        1.,  1.])

In [405]:
XX_sel = XX*feature_vec.T
pd.DataFrame(XX_sel).head()
XX_sel.shape

(2900, 15)

In [406]:
pd.DataFrame(XX_sel).head()

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14
0,0.378474,0,0.270943,0.443766,0.392646,0.003111,0.267020,0.116202,0.122794,0.193149,0.156651,0.336056,0.241134,0.153588,0.197704
1,0.200536,0,0.119515,0.197437,0.021939,-0.233513,0.248644,0.280343,0.038769,0.491651,-0.068320,0.157244,0.122218,0.380837,0.371301
2,0.225285,0,0.021889,-0.191315,0.127197,0.225558,0.435042,0.429598,0.119509,0.136093,-0.009471,-0.081929,-0.137766,0.575581,0.095648
3,0.155252,-0,0.491094,-0.169611,0.217508,0.028881,0.383180,0.135157,-0.049906,0.268499,0.394627,0.294663,0.359853,0.101031,0.176065
4,0.078659,0,0.044873,-0.174418,0.282761,0.143663,-0.151343,0.351289,0.022262,0.252626,0.415244,0.271127,0.513780,-0.056394,0.336774


In [407]:
Xtmp = np.zeros((XX_sel.shape[0],int(feature_vec.sum())))
Xtmp.shape

(2900, 14)

In [409]:
j = 0
for i in range(XX_sel.shape[1]):
    if XX_sel[:,i].sum() != 0:
        tmp.append(XX_sel[:,i])
        Xtmp[:,j] = XX_sel[:,i]
        j+=1
pd.DataFrame(Xtmp).head()

,0,1,2,3,4,5,6,7,8,9,10,11,12,13
0,0.378474,0.270943,0.443766,0.392646,0.003111,0.267020,0.116202,0.122794,0.193149,0.156651,0.336056,0.241134,0.153588,0.197704
1,0.200536,0.119515,0.197437,0.021939,-0.233513,0.248644,0.280343,0.038769,0.491651,-0.068320,0.157244,0.122218,0.380837,0.371301
2,0.225285,0.021889,-0.191315,0.127197,0.225558,0.435042,0.429598,0.119509,0.136093,-0.009471,-0.081929,-0.137766,0.575581,0.095648
3,0.155252,0.491094,-0.169611,0.217508,0.028881,0.383180,0.135157,-0.049906,0.268499,0.394627,0.294663,0.359853,0.101031,0.176065
4,0.078659,0.044873,-0.174418,0.282761,0.143663,-0.151343,0.351289,0.022262,0.252626,0.415244,0.271127,0.513780,-0.056394,0.336774


In [410]:
XXXn = Xtmp[:X.shape[0],:]
XXXtn = Xtmp[X.shape[0]:,:]

In [411]:
XXXn.shape, XXXtn.shape

((900, 14), (2000, 14))

following is just an elegant way to remove zero columns, but not working yet.

In [387]:
XXX = pd.DataFrame(XX_sel)
XXX.head()

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14
0,0.378474,0,0.270943,0.443766,0.392646,0.003111,0.267020,0,0,0.193149,0,0.336056,0.241134,0.153588,0.197704
1,0.200536,0,0.119515,0.197437,0.021939,-0.233513,0.248644,0,0,0.491651,-0,0.157244,0.122218,0.380837,0.371301
2,0.225285,0,0.021889,-0.191315,0.127197,0.225558,0.435042,0,0,0.136093,-0,-0.081929,-0.137766,0.575581,0.095648
3,0.155252,-0,0.491094,-0.169611,0.217508,0.028881,0.383180,0,-0,0.268499,0,0.294663,0.359853,0.101031,0.176065
4,0.078659,0,0.044873,-0.174418,0.282761,0.143663,-0.151343,0,0,0.252626,0,0.271127,0.513780,-0.056394,0.336774


In [316]:
selection = XXX.ix[:,XXX.sum()!=0]

In [388]:
selection = pd.DataFrame(selection)
selection.head()

,0,2,5,6,9,11,12,13,14
0,0.378474,0.270943,0.003111,0.267020,0.193149,0.336056,0.241134,0.153588,0.197704
1,0.200536,0.119515,-0.233513,0.248644,0.491651,0.157244,0.122218,0.380837,0.371301
2,0.225285,0.021889,0.225558,0.435042,0.136093,-0.081929,-0.137766,0.575581,0.095648
3,0.155252,0.491094,0.028881,0.383180,0.268499,0.294663,0.359853,0.101031,0.176065
4,0.078659,0.044873,0.143663,-0.151343,0.252626,0.271127,0.513780,-0.056394,0.336774


In [391]:
#XXXn = selection.ix[:X.shape[0],:]
#XXXtn = selection.ix[X.shape[0]:,:]
#XXXn

,0,2,5,6,9,11,12,13,14
0,0.378474,0.270943,0.003111,0.267020,0.193149,0.336056,0.241134,0.153588,0.197704
1,0.200536,0.119515,-0.233513,0.248644,0.491651,0.157244,0.122218,0.380837,0.371301
2,0.225285,0.021889,0.225558,0.435042,0.136093,-0.081929,-0.137766,0.575581,0.095648
3,0.155252,0.491094,0.028881,0.383180,0.268499,0.294663,0.359853,0.101031,0.176065
4,0.078659,0.044873,0.143663,-0.151343,0.252626,0.271127,0.513780,-0.056394,0.336774
5,0.055561,0.074239,-0.029112,0.335071,0.438620,0.384067,0.144560,-0.012472,0.362151
6,0.094546,0.191410,0.270349,0.160080,0.222098,-0.055091,-0.042849,0.311984,-0.027426
7,0.384194,0.032962,0.374267,0.248772,0.095538,0.254150,0.110286,0.210666,0.250882
8,0.266085,0.250970,0.164389,0.109194,0.272084,0.297561,0.062957,0.199469,0.070397
9,0.286105,0.135114,-0.083840,0.245651,0.577378,0.014878,0.361297,-0.053914,0.091888


# [Linear Regression]

In [116]:
regressor = sklin.LinearRegression()
regressor.fit(Xn,y)
eval_pred(regressor,Xn,y)

C-V score = 41.4333383848 +/- 5.64233820591


In [121]:
regressor.fit(Xn,y)
#this function print parameter of regression
"""
feature_names = ['x1','x2', 'x3', 'x4', 'x5', 'x6', 'x7', 'x8', 'x9', 'x10', 'x11', 'x12', 'x13', 'x14', 'x15']
print('{0:>10} {1:+.4f}'.format('intercept', regressor.intercept_))
for feature_name, coef in zip(feature_names, regressor.coef_):
    print('{0:>10} {1:+.4f}'.format(feature_name, coef))
"""

 intercept -46.9222
        x1 +43.7654
        x2 +16.7069
        x3 -10.5726
        x4 +31.9376
        x5 -2.8208
        x6 +56.0321
        x7 +92.1729
        x8 +27.4054
        x9 +11.7272
       x10 -38.5210
       x11 +24.8832
       x12 +57.8268
       x13 -55.1408
       x14 +120.0936
       x15 +58.2473


In [122]:
Ypred = regressor.predict(Xtn)

In [123]:
#pd.DataFrame(Ypred).head()

,0
0,8.346059
1,-31.383995
2,65.746103
3,18.816375
4,28.504684


In [124]:
Ypred.shape

(2000,)

In [125]:
sub = pd.read_csv("sample.csv")
sub['y'] = Ypred
sub.head()

,Id,y
0,900,8.346059
1,901,-31.383995
2,902,65.746103
3,903,18.816375
4,904,28.504684


In [97]:
sub.to_csv('test_submission_linear_regression.csv', index = False)

In [179]:
#this is just a test of GridSearch
regressor_ridge = sklin.Ridge()
param_grid = {'alpha': np.linspace(0, 100, 10)}
neg_scorefun = skmet.make_scorer(lambda x, y: -score(x, y))  # Note the negative sign.
grid_search = skgs.GridSearchCV(regressor_ridge, param_grid, scoring=neg_scorefun, cv=5)
grid_search.fit(Xn, y)

GridSearchCV(cv=5, error_score='raise',
       estimator=Ridge(alpha=1.0, copy_X=True, fit_intercept=True, max_iter=None,
   normalize=False, random_state=None, solver='auto', tol=0.001),
       fit_params={}, iid=True, n_jobs=1,
       param_grid={'alpha': array([   0.     ,   11.11111,   22.22222,   33.33333,   44.44444,
         55.55556,   66.66667,   77.77778,   88.88889,  100.     ])},
       pre_dispatch='2*n_jobs', refit=True, scoring=make_scorer(<lambda>),
       verbose=0)

In [180]:
#this is just a test of GridSearch
best = grid_search.best_estimator_
print(best)
print('best score =', -grid_search.best_score_)

Ridge(alpha=0.0, copy_X=True, fit_intercept=True, max_iter=None,
   normalize=False, random_state=None, solver='auto', tol=0.001)
best score = 41.5340930041


Linear Regression has very poor result, try nonlinear method:

# [Kernel Method]

In [117]:
from sklearn.svm import SVR
from sklearn.kernel_ridge import KernelRidge

In [118]:
# Fit regression model
svr_rbf = SVR(kernel='rbf', C=1e3, gamma=3)
svr_lin = SVR(kernel='linear', C=1e3)
svr_poly = SVR(kernel='poly', C=1e3, degree=10)

In [395]:
eval_pred(svr_rbf,XXXn,y)

C-V score = 41.5475557515 +/- 4.66660031055


In [396]:
eval_pred(svr_lin,Xn_sel,y)

C-V score = 42.3222403531 +/- 5.3412707062


In [397]:
eval_pred(svr_poly,Xn_sel,y)

C-V score = 53.7160131841 +/- 6.70558184339


In [70]:
y_rbf = svr_rbf.fit(Xn, y).predict(Xtn)
y_lin = svr_lin.fit(Xn, y).predict(Xtn)
y_poly = svr_poly.fit(Xn, y).predict(Xtn)

In [210]:
#plt.scatter(Xn[:,0], y, c='k', label='data')
#plt.hold('on')
#plt.plot(Xtn[:,0], y_rbf, c='g', label='RBF model')
#plt.plot(Xtn[:,0], y_lin, c='r', label='Linear model')
#plt.plot(Xn, y_poly, c='b', label='Polynomial model')
#plt.xlabel('data')
#plt.ylabel('target')
#plt.title('Support Vector Regression')
#plt.legend()
#plt.show()

In [71]:
sub = pd.read_csv("sample.csv")
sub['y'] = y_rbf
#sub.head()

In [177]:
sub.to_csv('test_submission_rbf_regression.csv', index = False)

# use GridSearch to find optimal gamma and C/alpha

In [149]:
from sklearn.grid_search import GridSearchCV
from sklearn.learning_curve import learning_curve
from sklearn.kernel_ridge import KernelRidge

In [150]:
svr_rbf_grid = GridSearchCV(SVR(kernel='rbf', gamma=0.1), cv=5,
                   param_grid={"C": [1e0, 1e1, 1e2, 1e3],
                               "gamma": np.logspace(-2, 2, 5)})

In [151]:
kr = GridSearchCV(KernelRidge(kernel='rbf', gamma=0.1), cv=5,
                  param_grid={"alpha": [1e0, 0.1, 1e-2, 1e-3],
                              "gamma": np.logspace( -2, 4,10)})

In [386]:
eval_pred(svr_rbf_grid,Xn_sel,y)

C-V score = 37.4855406272 +/- 4.81333250064


In [398]:
eval_pred(kr,XXXn,y)

C-V score = 34.7266310399 +/- 4.57441115477


In [47]:
kr.fit(Xn,y)
y_kr = kr.predict(Xtn)

In [48]:
sub = pd.read_csv("sample.csv")
sub['y'] = y_kr
sub.head()
sub.to_csv('test_submission_kr_regression.csv', index = False)

As can be seen above, SVR does not provide better result than linear regression

# [Decision Tree Regression]

In [126]:
from sklearn.tree import DecisionTreeRegressor

regr_1 = DecisionTreeRegressor(max_depth=2)
regr_2 = DecisionTreeRegressor(max_depth=20)
#regr_1.fit(Xn, y)
#regr_2.fit(Xn, y)

In [120]:
eval_pred(regr_1,Xn,y)

C-V score = 49.3719088998 +/- 6.44466425961


In [122]:
eval_pred(regr_2,Xn,y)

C-V score = 60.976176388 +/- 7.05984992075


# [AdaBoostRegressor]

In [128]:
from sklearn.ensemble import AdaBoostRegressor

In [139]:
Ada_1 = AdaBoostRegressor(DecisionTreeRegressor(max_depth=10),
                          n_estimators=300, random_state=rng)
Ada_2 = AdaBoostRegressor(sklin.LinearRegression(),
                          n_estimators=300, random_state=rng)
Ada_3 = AdaBoostRegressor(SVR(kernel='rbf', C=1e3, gamma=3),
                          n_estimators=300, random_state=rng)
Ada_4 = AdaBoostRegressor(SVR(kernel='linear', C=1e3),
                          n_estimators=300, random_state=rng)
Ada_5 = AdaBoostRegressor(SVR(kernel='poly', C=1e3, degree=7),
                          n_estimators=300, random_state=rng)
Ada_6 = AdaBoostRegressor(KernelRidge(kernel='rbf', gamma=0.3,alpha = 1e-2),
                          n_estimators=300, random_state=rng)

#kr = GridSearchCV(KernelRidge(kernel='rbf', gamma=0.1), cv=5,
#                  param_grid={"alpha": [1e0, 0.1, 1e-2, 1e-3],
#                              "gamma": np.logspace( -2, 4,10)})

In [130]:
eval_pred(Ada_1,Xn,y)

C-V score = 40.4214009241 +/- 5.57970312992


In [132]:
eval_pred(Ada_2,Xn,y)

C-V score = 43.848258774 +/- 4.44647526475


In [399]:
eval_pred(Ada_3,XXXn,y)

C-V score = 41.0371566306 +/- 5.91583587865


In [135]:
eval_pred(Ada_4,Xn,y)

C-V score = 61.5146243022 +/- 6.47354425106


In [140]:
eval_pred(Ada_5,Xn,y)

C-V score = 53.5671108783 +/- 6.59248001956


In [141]:
eval_pred(Ada_6,Xn,y)

C-V score = 40.5967636895 +/- 4.85634617326


# [ARDregression]

In [88]:
from sklearn.linear_model import ARDRegression, LinearRegression

In [89]:
ARD = ARDRegression(compute_score=True)
eval_pred(ARD,Xn,y)

C-V score = 41.5098649897 +/- 5.61030053061


In [216]:
LinReg = LinearRegression()
eval_pred(ARD,Xn,y)

C-V score = 41.6580921227 +/- 4.61007520981


# [Try lasso and polynomialFeature]

In [143]:
from sklearn import linear_model

In [144]:
lasso = linear_model.LassoLars(alpha=0.01)
eval_pred(lasso,Xn,y)

C-V score = 41.4378822643 +/- 5.68444088027


In [145]:
from sklearn.linear_model import Perceptron
from sklearn.preprocessing import PolynomialFeatures

In [400]:
XXXn_tf = PolynomialFeatures(interaction_only=True).fit_transform(XXXn)
Xtn_tf = PolynomialFeatures(interaction_only=True).fit_transform(Xtn)

In [237]:
#percep = Perceptron(fit_intercept=False, n_iter=10, shuffle=False).fit(Xn_tf, y)
eval_pred(lasso,Xn_tf,y)

C-V score = 37.7106294753 +/- 3.75400247959


In [238]:
eval_pred(svr_rbf,Xn_tf,y)

C-V score = 38.9624849892 +/- 5.35912963427


In [1]:
from sklearn import linear_model

In [2]:
lm_lasso = linear_model.Lasso(alpha=0.1)

In [22]:
eval_pred(lm_lasso,Xn,y)

C-V score = 41.5857177687 +/- 4.74568425394


In [413]:
eval_pred(kr,XXXn,y)

C-V score = 33.7728274446 +/- 5.04607670553


In [414]:
eval_pred(svr_rbf_grid,XXXn,y)

C-V score = 34.2232761883 +/- 5.04737604322


In [156]:
svr_rbf_grid.fit(Xn_tf,y)

GridSearchCV(cv=5, error_score='raise',
       estimator=SVR(C=1.0, cache_size=200, coef0=0.0, degree=3, epsilon=0.1, gamma=0.1,
  kernel='rbf', max_iter=-1, shrinking=True, tol=0.001, verbose=False),
       fit_params={}, iid=True, n_jobs=1,
       param_grid={'gamma': array([  1.00000e-02,   1.00000e-01,   1.00000e+00,   1.00000e+01,
         1.00000e+02]), 'C': [1.0, 10.0, 100.0, 1000.0]},
       pre_dispatch='2*n_jobs', refit=True, scoring=None, verbose=0)

In [158]:
Ypred = svr_rbf_grid.predict(Xtn_tf)

In [159]:
sub = pd.read_csv("sample.csv")
sub['y'] = Ypred
sub.head()
sub.to_csv('test_submission_svr_rbf_grid.csv', index = False)

In [271]:
pd.DataFrame(Xtn_tf).head()

,0,1,2,3,4,5,6,7,8,9,...,111,112,113,114,115,116,117,118,119,120
0,1,0.292418,0.420133,-0.095690,0.043988,0.270180,-0.072993,0.234864,-0.101510,0.598943,...,0.006005,0.002601,0.000064,0.006553,0.023746,0.000581,0.059818,0.000252,0.025915,0.000634
1,1,-0.014159,-0.123055,0.251215,0.235878,0.144004,-0.068836,0.075333,0.341366,0.571733,...,0.000205,0.056164,0.035218,-0.005228,0.000579,0.000363,-0.000054,0.099512,-0.014774,-0.009264
2,1,0.293873,0.138530,-0.020185,0.200825,-0.007854,0.414606,0.444049,0.441633,0.230808,...,-0.022727,0.083161,0.040073,0.056176,-0.023033,-0.011099,-0.015559,0.040613,0.056933,0.027434
3,1,0.380262,-0.050227,0.243550,-0.026532,-0.257482,0.147521,0.149179,0.258393,0.161242,...,0.108099,0.196190,0.096503,0.058306,0.073601,0.036203,0.021874,0.065706,0.039699,0.019527
4,1,-0.002093,0.334725,0.285952,-0.047922,0.257012,0.435998,-0.102000,0.557526,-0.027146,...,-0.000812,0.000379,-0.000601,-0.000727,-0.028423,0.045102,0.054580,-0.021028,-0.025447,0.040380


polynomialFeature does not work

# [Feature Selection + Kernel]

In [1]:
from sklearn.feature_selection import VarianceThreshold

In [83]:
sel = VarianceThreshold(threshold=(0.0327))

In [84]:
Xsel = sel.fit_transform(Xn)

In [85]:
pd.DataFrame(Xsel).head()

,0,1,2,3,4,5,6,7
0,0.378474,0.443766,0.003111,0.116202,0.122794,0.241134,0.153588,0.197704
1,0.200536,0.197437,-0.233513,0.280343,0.038769,0.122218,0.380837,0.371301
2,0.225285,-0.191315,0.225558,0.429598,0.119509,-0.137766,0.575581,0.095648
3,0.155252,-0.169611,0.028881,0.135157,-0.049906,0.359853,0.101031,0.176065
4,0.078659,-0.174418,0.143663,0.351289,0.022262,0.513780,-0.056394,0.336774


In [80]:
eval_pred(svr_rbf,Xsel,y)

C-V score = 33.824402195 +/- 4.33528465247


In [81]:
eval_pred(svr_lin,Xsel,y)

C-V score = 42.0567764526 +/- 5.664819972


In [82]:
eval_pred(svr_poly,Xsel,y)

C-V score = 53.7160131813 +/- 6.70558184327


In [87]:
eval_pred(regressor,Xsel,y)

C-V score = 46.0766948945 +/- 6.2363446876


In [90]:
eval_pred(ARD,Xsel,y)

C-V score = 46.1344526791 +/- 6.15748543764


In [153]:
eval_pred(svr_rbf_grid,Xsel,y)

C-V score = 43.9930545821 +/- 5.84936505436


# feature selection

In [94]:
from scipy import linalg
from sklearn.linear_model import (RandomizedLasso, lasso_stability_path,
                                  LassoLarsCV)
from sklearn.feature_selection import f_regression
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import auc, precision_recall_curve
from sklearn.ensemble import ExtraTreesRegressor
from sklearn.utils.extmath import pinvh
from sklearn.utils import ConvergenceWarning
import warnings

In [92]:
def mutual_incoherence(X_relevant, X_irelevant):
    """Mutual incoherence, as defined by formula (26a) of [Wainwright2006].
    """
    projector = np.dot(np.dot(X_irelevant.T, X_relevant),
                       pinvh(np.dot(X_relevant.T, X_relevant)))
    return np.max(np.abs(projector).sum(axis=1))